In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec 18 11:41:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install datasets==1.18.3
!pip install transformers==4.24.0
!pip install tokenizers==0.13.2
!pip install huggingface_hub==0.11
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer
!apt install git-lfs
!pip install wandb
!pip install pandas

from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [6]:
from huggingface_hub.hf_api import HfFolder

HfFolder.save_token('HF_TOKEN')

!huggingface-cli whoami
!wandb login --cloud HF_TOKEN

henilp105
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# load IIITH ASR dataset

In [4]:
import json
import csv
import shutil
import torchaudio
import librosa
import numpy as np
from datasets import load_dataset, load_metric, Audio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import re
import json
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor ,Wav2Vec2Processor

In [5]:
SEED = 13
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def dataset_folder(foldername,setname):
  a=[]
  with open(f'{foldername}/data.json') as f:
        data = json.load(f)
        for i in data:
          if setname in i['audioFilename']:
            a.append(i)

  with open(f'{foldername}/data_{setname}.json', 'w') as fp:
      json.dump(a, fp)

  df = pd.read_json(f"{foldername}/data_{setname}.json")
  df['sentence'] = df['text']
  print("dataset length:",len(df))
  train_df, test_df = train_test_split(df, test_size=0.3, random_state = SEED)
  !rm -rf test train
  !mkdir test train
  print("train dataset: ",len(train_df))
  print("test dataset: ",len(test_df))
  train_df['path'] = "train/"+train_df.audioFilename.str.slice(12)
  test_df['path'] = "test/"+test_df.audioFilename.str.slice(12)
  train_df.to_csv("train/metadata.csv")
  !cp -r {foldername}/audio/{setname}/*.wav train/
  !cp -r {foldername}/audio/{setname}/*.wav test/
  test_df.to_csv("test/metadata.csv", index = False)

def remove_special_characters(batch):
  text = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower() + " "
  text = text.replace("\\n","\n")
  text = ' '.join(text.split())
  text = re.sub(r'''%'''," శాతం ", text)
  text = re.sub(r'''(/|-|_)'''," ", text)
  text = re.sub("ై","ై", text)
  text = text.strip()
  batch['sentence'] = text
  return batch

def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

def prepare_dataset(batch):
    audio = batch["audio"]
    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

def speech_to_array(batch):
    file_path = batch['path']
    speech_array, _ = torchaudio.load(file_path)
    speech_array = speech_array[0].numpy()
    sampling_rate = 16_000
    a = dict()
    a['path'] = file_path
    a['array'] = speech_array
    a['sampling_rate'] = sampling_rate
    batch['audio'] = a
    return batch


In [16]:
dataset_folder("f","set_2")
common_voice_train = load_dataset("train", data_dir="train")['train']
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_train = common_voice_train.map(speech_to_array)
common_voice_train = common_voice_train.remove_columns(["channel", "age","audioFilename","text"])

common_voice_test = load_dataset("test", data_dir="test")['train']
common_voice_test = common_voice_test.map(remove_special_characters)
common_voice_test = common_voice_test.map(speech_to_array)
common_voice_test = common_voice_test.remove_columns(["channel", "age","audioFilename","text"])

dataset length: 2000
train dataset:  1400
test dataset:  600


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/train-b6a694798e802f8e/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/test-a27ec00413b8a746/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [17]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)
    
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
repo_name = "wav2vec2-large-xls-r-300m-telugu-asr"
tokenizer.push_to_hub(repo_name)
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [18]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
wer_metric = load_metric("wer")

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer
# from transformers.trainer_utils import get_last_checkpoint
# last_checkpoint = get_last_checkpoint("wav2vec2-large-xls-r-300m-te")

model = Wav2Vec2ForCTC.from_pretrained(
    "henilp105/wav2vec2-large-xls-r-300m-telugu-asr",
    attention_dropout=0.0,
    ignore_mismatched_sizes=True,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

model.freeze_feature_extractor()

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,  
  gradient_accumulation_steps=2,  # 2 
  evaluation_strategy="steps",
  num_train_epochs=30,  # 30 max    
  gradient_checkpointing=True,
  fp16=True,
  save_steps=800,   # save 800 eval 400 epochs 30   
  eval_steps=200,
  logging_steps=200,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

train_result = trainer.train()
trainer.push_to_hub()

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at henilp105/wav2vec2-large-xls-r-300m-telugu-asr and are newly initialized because the shapes did not match:
- lm_head.weight: found shape torch.Size([64, 1024]) in the checkpoint and torch.Size([68, 1024]) in the model instantiated
- lm_head.bias: found shape torch.Size([64]) in the checkpoint and torch.Size([68]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1635: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(
Cloning https://huggingface.co/henilp105/wav2vec2-large-xls-r-300m-telugu-asr into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.0k/1.18G [00:00<?, ?B/s]

Download file runs/Dec18_07-40-53_4d9292647b92/events.out.tfevents.1671349743.4d9292647b92.75.0: 100%|########…

Download file runs/Dec17_14-11-16_b6f469d27bbe/1671286341.8529754/events.out.tfevents.1671286341.b6f469d27bbe.…

Download file runs/Dec18_07-40-53_4d9292647b92/1671349743.019885/events.out.tfevents.1671349743.4d9292647b92.7…

...

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
200,7.036500,3.385284,1.000000
400,1.454600,1.027498,0.809868
600,0.536700,1.022744,0.786444
800,0.347400,1.099835,0.788188


The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
The following columns in the evaluation set don't have a corr

Step,Training Loss,Validation Loss,Wer
200,7.036500,3.385284,1.000000
400,1.454600,1.027498,0.809868
600,0.536700,1.022744,0.786444
800,0.347400,1.099835,0.788188
1000,0.242600,1.192042,0.792175
1200,0.181800,1.252438,0.787192
1400,0.146000,1.326313,0.777473
1600,0.126300,1.312966,0.758535
1800,0.108000,1.335252,0.746324
2000,0.092100,1.387060,0.761276


The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
The following columns in the evaluation set don't have a corr

In [ ]:
metrics = train_result.metrics
max_train_samples = len(common_voice_train)
metrics["train_samples"] = min(max_train_samples, len(common_voice_train))

# trainer.save_model()

# trainer.log_metrics("train", metrics)
# trainer.save_metrics("train", metrics)
# trainer.save_state()
# trainer.push_to_hub()
# from transformers.trainer_utils import get_last_checkpoint
# last_checkpoint = get_last_checkpoint("wav2vec2-large-xls-r-300m-te")
# print(last_checkpoint)
# train_result = trainer.train(resume_from_checkpoint=last_checkpoint)